In [33]:
_ = ! pip install web3

In [81]:
import os, sys

from datetime import datetime, timedelta
import time

import requests, json
from web3 import Web3, HTTPProvider

from eth_utils import to_bytes
from eth_keys import keys
from eth_account import Account
from eth_account.messages import SignableMessage

In [119]:
w3 = Web3(HTTPProvider("https://rpc.ankr.com/eth/1e5f9fd3e49ecec581e00815ea1eb41bc459248fd3dcf67716ff3237cc3ab68b"))
# https://1rpc.io/eth

In [120]:
source_token = "0x1f9840a85d5af5bf1d1762f925bdaddc4201f984" # UniSwap
destination_token = "0xdac17f958d2ee523a2206206994597c13d831ec7" # USDT

source_amount = 1000000000000000000
slippage = int(0.05 * 10000) # Convert to bps

In [121]:
eth_wallet = Account.from_key("e5ecbcfc71ad24a2131953ef9a66e4b7ed8ca9a10ffd92104e106fab1f0af4f7")

In [122]:
# 0.  One fresh quote – DO NOT fetch another one before sending
quote  = requests.get(
    "https://api.0x.org/swap/permit2/quote",
    params={
        "chainId": 1,
        "sellToken": source_token,
        "buyToken":  destination_token,
        "sellAmount": source_amount,
        "taker": eth_wallet.address,
        "slippageBps": slippage,
    },
    headers={
        "0x-version" : "v2",
        "0x-api-key" : "50cbb531-4fda-4464-acb4-5e6b5bf0e2d6"
    },
    timeout=10,
).json()

In [123]:
ERC20_ABI: list[dict] = [
    {
        "constant": True,
        "inputs": [{"name": "owner", "type": "address"}],
        "name": "balanceOf",
        "outputs": [{"name": "balance", "type": "uint256"}],
        "type": "function",
    },
    {
        "constant": True,
        "inputs": [],
        "name": "decimals",
        "outputs": [{"name": "", "type": "uint8"}],
        "type": "function",
    },
]

erc20 = w3.eth.contract(address=Web3.to_checksum_address(source_token), abi=ERC20_ABI)

sell_amount   = int(quote["sellAmount"])
permit_amount = int(quote["permit2"]["eip712"]["message"]["permitted"]["amount"])
owner_balance = int(erc20.functions.balanceOf(eth_wallet.address).call())

print ("sellAmount        :", sell_amount)
print ("permit.amount     :", permit_amount)
print ("wallet UNI balance:", owner_balance)
print ()
print ("gas fee:", (1.0e-18) * int(quote['transaction']['gas']) * int(quote['transaction']['gasPrice']), "ETH")

sellAmount        : 1000000000000000000
permit.amount     : 1000000000000000000
wallet UNI balance: 57116026654249141437

gas fee: 0.00020617792014352503 ETH


In [124]:
# 1) Sign the raw 32-byte digest that 0x gave you
digest = bytes.fromhex(quote["permit2"]["hash"][2:])

sig_obj   = keys.PrivateKey(eth_wallet.key).sign_msg_hash(digest)
sig_bytes = sig_obj.to_bytes()

# Normalise v to 27/28 (some eth-keys return 0/1)
if sig_bytes[-1] in (0, 1):
    sig_bytes = sig_bytes[:-1] + bytes([sig_bytes[-1] + 27])

assert isinstance(sig_bytes, (bytes, bytearray)), "sig must be bytes, not str"
assert len(sig_bytes) == 65, f"signature is {len(sig_bytes)} bytes, must be 65"

# (optional but recommended) prove we signed with the right key
rec = sig_obj.recover_public_key_from_msg_hash(digest).to_checksum_address()
assert rec.lower() == eth_wallet.address.lower(), "signer mismatch"

# 2) Patch the **exact** placeholder, not “last 97 bytes”
data_hex = quote["transaction"]["data"]
data     = bytes.fromhex(data_hex[2:])

ZERO97 = b"\x00" * (32 + 65)
off = data.rfind(ZERO97)
if off == -1:
    raise ValueError("Permit2 placeholder not found – API layout changed?")

patched = (
    data[:off]
    + (65).to_bytes(32, "big")      # length word
    + sig_bytes                     # r||s||v (65 bytes)
    + data[off + len(ZERO97):]
)

# 3) Sanity checks that kill InvalidSignatureLength() once and for all
len_word = int.from_bytes(patched[off:off+32], "big")
print("patch @", off, "len_word =", len_word)
print("len_word_hex:", patched[off:off+32].hex())
print("first 8 bytes of sig r:", patched[off+32:off+40].hex())

assert len_word == 65, "length word still not 65"
# ensure no second zero placeholder after ours
assert patched.rfind(ZERO97) < off, "another 97-zero block remains after injected signature"

calldata = "0x" + patched.hex()

offset  : 1025
lenword : 65
lenword hex: 0000000000000000000000000000000000000000000000000000000000000041
next 32B: 712cc9ba ...


In [125]:
# 4. Build, sign, send
tx: dict = {
    "chainId":  1,
    "to":       Web3.to_checksum_address(quote["transaction"]["to"]),
    "data":     calldata,
    "value":    int(quote["transaction"]["value"]),
    "gas":      int(quote["transaction"]["gas"]),
    "nonce":    w3.eth.get_transaction_count(eth_wallet.address),
}

# Prefer EIP-1559 if present
if ("maxFeePerGas" in quote["transaction"]) and ("maxPriorityFeePerGas" in quote["transaction"]):
    tx["type"] = 2
    tx["maxFeePerGas"] = int(quote["transaction"]["maxFeePerGas"])
    tx["maxPriorityFeePerGas"] = int(quote["transaction"]["maxPriorityFeePerGas"])
else:
    tx["gasPrice"] = int(quote["transaction"]["gasPrice"])

# Pass through optional fields
if "accessList" in quote["transaction"]:
    tx["accessList"] = quote["transaction"]["accessList"]

# Only for eth_call / estimateGas preflight
tx = dict(tx, **{"from": eth_wallet.address})

tx_signed = eth_wallet.sign_transaction(tx)
tx_hash = w3.eth.send_raw_transaction(tx_signed.raw_transaction)

In [126]:
# 5. Check result
tx_hex = w3.to_hex(tx_hash)

print("↗ broadcast :", tx_hex)
print("⏳ waiting …")
print("✅ status   :", w3.eth.wait_for_transaction_receipt(tx_hash).status)  # 1 = success

↗ broadcast : 0xee8623e5bb52b1547b9fd332b4d3ea79171b488d4bf4cb0931ce9b50e89450d9
⏳ waiting …
✅ status   : 0


In [127]:
print ("https://eth.blockscout.com/tx/" + tx_hex)

https://eth.blockscout.com/tx/0xee8623e5bb52b1547b9fd332b4d3ea79171b488d4bf4cb0931ce9b50e89450d9
